#### Lora实战

##### step 1. 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM,DataCollatorForSeq2Seq, TrainingArguments, Trainer

2023-10-15 09:27:30.230424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Step 2. 加载数据集

In [2]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")

In [3]:
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

#### Step 3. 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("../../models/Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='../../models/Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [6]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Loading cached processed dataset at /home/ev2li/workspace/deep-learning/example/llm/data/alpaca_data_zh/cache-0adbe3cfce44d3d6.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

#### Step 4. 创建模型

In [10]:
model = AutoModelForCausalLM.from_pretrained("../../models/Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [11]:
for name, param in model.named_parameters():
    print(name)

transformer.word_embeddings.weight
transformer.word_embeddings_layernorm.weight
transformer.word_embeddings_layernorm.bias
transformer.h.0.input_layernorm.weight
transformer.h.0.input_layernorm.bias
transformer.h.0.self_attention.query_key_value.weight
transformer.h.0.self_attention.query_key_value.bias
transformer.h.0.self_attention.dense.weight
transformer.h.0.self_attention.dense.bias
transformer.h.0.post_attention_layernorm.weight
transformer.h.0.post_attention_layernorm.bias
transformer.h.0.mlp.dense_h_to_4h.weight
transformer.h.0.mlp.dense_h_to_4h.bias
transformer.h.0.mlp.dense_4h_to_h.weight
transformer.h.0.mlp.dense_4h_to_h.bias
transformer.h.1.input_layernorm.weight
transformer.h.1.input_layernorm.bias
transformer.h.1.self_attention.query_key_value.weight
transformer.h.1.self_attention.query_key_value.bias
transformer.h.1.self_attention.dense.weight
transformer.h.1.self_attention.dense.bias
transformer.h.1.post_attention_layernorm.weight
transformer.h.1.post_attention_layernor

#### Lora
PEFT Step 1. 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
# config = LoraConfig(task_type=TaskType.CAUSAL_LM)
config = LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=["query_key_value","dense_4h_to_h"])
# config = LoraConfig(task_type=TaskType.CausalLM, target_modules=[".*\.1.*query_key_value"], modules_to_save=[])
config

PEFT Step 2. 创建模型

In [ ]:
model = get_peft_model(model, config)

In [ ]:
config

In [ ]:
model

In [ ]:
for name, param in model.named_parameters():
    print(name)

In [ ]:
model.print_trainable_parameters()

#### Step 5. 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

#### Step 6. 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

#### Step 7. 模型训练

In [ ]:
trainer.train()

#### Step 8. 模型推理

In [ ]:
model = model.cuda()
ipt = tokenizer("Human:{}\n{}".format("考试有哪些技巧?", "").strip() + "\n\nAssistant: ", return_tensors="pt").to("cuda")
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)